In [1]:
import pandas as pd
v = pd.read_csv("/Users/nsehgal/Documents/Data Mining/dataset/TrainFeatures.csv")
v2 = pd.read_csv("/Users/nsehgal/Documents/Data Mining/dataset/TestFeatures.csv")

In [2]:
v.dtypes

user_id        object
video_id       object
session        object
new_speed     float64
old_speed     float64
new_time      float64
old_time      float64
event_type     object
event_time     object
dtype: object

In [3]:
import pandas as pd
from datetime import date
from datetime import datetime
v["event_time"] = pd.to_datetime(v["event_time"])   #Changing Dates as Dates types 
v2["event_time"] = pd.to_datetime(v2["event_time"])

In [4]:
v.dtypes

user_id               object
video_id              object
session               object
new_speed            float64
old_speed            float64
new_time             float64
old_time             float64
event_type            object
event_time    datetime64[ns]
dtype: object

In [6]:
v['event_type'].value_counts()  #This is used to define weights later for how many user 
# has played, paused, seek or stop each video.

play_video            858430
pause_video           398478
seek_video            332388
load_video            330342
stop_video            115655
speed_change_video     13373
Name: event_type, dtype: int64

In [9]:
#Here, each user is grouped by video and event type. Further, count for each action for each video is calculated.
b= v.groupby(["user_id","video_id","event_type"]).agg({'event_type':['count']})
b1= v2.groupby(["user_id","video_id","event_type"]).agg({'event_type':['count']})

In [10]:
b.head()

event_type
                                                                                  count
user_id                          video_id                         event_type           
0005266eb37020aaa9d87eccc6d051b3 0fca720c83a20f7f881fadfa031593a6 load_video          1
                                 120c6b171f9909315d6abf6c2c047c28 load_video          1
                                 12d80cac55fe5eac6b64ed67c60b2312 load_video          2
                                 29f52e6f78f5f97b9b1c917381d1c3b3 load_video          1
                                 2eae274705ed306dae506f5ccd16f2c7 load_video          5

In [11]:
b= v.groupby(["user_id","video_id","event_type"]).agg({'event_type':['count']}).reset_index()
b1= v2.groupby(["user_id","video_id","event_type"]).agg({'event_type':['count']}).reset_index()
b.columns = b.columns.get_level_values(0)
b1.columns = b1.columns.get_level_values(0)
b.head(5)

,user_id,video_id,event_type,event_type
0,0005266eb37020aaa9d87eccc6d051b3,0fca720c83a20f7f881fadfa031593a6,load_video,1
1,0005266eb37020aaa9d87eccc6d051b3,120c6b171f9909315d6abf6c2c047c28,load_video,1
2,0005266eb37020aaa9d87eccc6d051b3,12d80cac55fe5eac6b64ed67c60b2312,load_video,2
3,0005266eb37020aaa9d87eccc6d051b3,29f52e6f78f5f97b9b1c917381d1c3b3,load_video,1
4,0005266eb37020aaa9d87eccc6d051b3,2eae274705ed306dae506f5ccd16f2c7,load_video,5


In [13]:
#Last two columns are renamed to handle index after resetting them.
b.columns =['user_id','video_id','event_type','type_count']
b1.columns =['user_id','video_id','event_type','type_count']

In [14]:
b.head(5)

,user_id,video_id,event_type,type_count
0,0005266eb37020aaa9d87eccc6d051b3,0fca720c83a20f7f881fadfa031593a6,load_video,1
1,0005266eb37020aaa9d87eccc6d051b3,120c6b171f9909315d6abf6c2c047c28,load_video,1
2,0005266eb37020aaa9d87eccc6d051b3,12d80cac55fe5eac6b64ed67c60b2312,load_video,2
3,0005266eb37020aaa9d87eccc6d051b3,29f52e6f78f5f97b9b1c917381d1c3b3,load_video,1
4,0005266eb37020aaa9d87eccc6d051b3,2eae274705ed306dae506f5ccd16f2c7,load_video,5


In [17]:
# Here wighted are assigned if a specific user has played, paused, seek or stop each video
def scoring(x):
    if x=='load_video':
        return 0*10
    if x=='play_video':
        return 0.5*10
    if x=='seek_video':
        return 0.23*10
    if x=='pause_video':
        return 0.19*10
    if x=='stop_video':
        return 0.07*10
    if x=='speed_change_video':
        return 0*10

 #Weighted score is calcuated depending on count of action * weight for each action
b['score'] = b['event_type'].apply(scoring)
b['weighted_score'] = b['score']*b['type_count']
b1['score'] = b1['event_type'].apply(scoring)
b1['weighted_score'] = b1['score']*b1['type_count']

In [18]:
b.head(10)

,user_id,video_id,event_type,type_count,score,weighted_score
0,0005266eb37020aaa9d87eccc6d051b3,0fca720c83a20f7f881fadfa031593a6,load_video,1,0.0,0.0
1,0005266eb37020aaa9d87eccc6d051b3,120c6b171f9909315d6abf6c2c047c28,load_video,1,0.0,0.0
2,0005266eb37020aaa9d87eccc6d051b3,12d80cac55fe5eac6b64ed67c60b2312,load_video,2,0.0,0.0
3,0005266eb37020aaa9d87eccc6d051b3,29f52e6f78f5f97b9b1c917381d1c3b3,load_video,1,0.0,0.0
4,0005266eb37020aaa9d87eccc6d051b3,2eae274705ed306dae506f5ccd16f2c7,load_video,5,0.0,0.0
5,0005266eb37020aaa9d87eccc6d051b3,310cb4c79ef141136b78053252398985,load_video,2,0.0,0.0
6,0005266eb37020aaa9d87eccc6d051b3,310cb4c79ef141136b78053252398985,pause_video,1,1.9,1.9
7,0005266eb37020aaa9d87eccc6d051b3,310cb4c79ef141136b78053252398985,play_video,1,5.0,5.0
8,0005266eb37020aaa9d87eccc6d051b3,310cb4c79ef141136b78053252398985,stop_video,1,0.7,0.7
9,0005266eb37020aaa9d87eccc6d051b3,3648f1f1247462252aefec619f2fae64,load_video,1,0.0,0.0


In [19]:
# index has to be reset after groupby
c= b.groupby(["user_id","video_id"]).agg({'weighted_score':['sum']}).reset_index()
c.columns=c.columns.get_level_values(0)

c1= b1.groupby(["user_id","video_id"]).agg({'weighted_score':['sum']}).reset_index()
c1.columns=c1.columns.get_level_values(0)
c.head(50)

,user_id,video_id,weighted_score
0,0005266eb37020aaa9d87eccc6d051b3,0fca720c83a20f7f881fadfa031593a6,0.0
1,0005266eb37020aaa9d87eccc6d051b3,120c6b171f9909315d6abf6c2c047c28,0.0
2,0005266eb37020aaa9d87eccc6d051b3,12d80cac55fe5eac6b64ed67c60b2312,0.0
3,0005266eb37020aaa9d87eccc6d051b3,29f52e6f78f5f97b9b1c917381d1c3b3,0.0
4,0005266eb37020aaa9d87eccc6d051b3,2eae274705ed306dae506f5ccd16f2c7,0.0
5,0005266eb37020aaa9d87eccc6d051b3,310cb4c79ef141136b78053252398985,7.6
6,0005266eb37020aaa9d87eccc6d051b3,3648f1f1247462252aefec619f2fae64,0.0
7,0005266eb37020aaa9d87eccc6d051b3,38d92329d49a9e2625c1dffe8503297b,0.0
8,0005266eb37020aaa9d87eccc6d051b3,455ee2a219965da7526e2a167853e3d9,0.0
9,0005266eb37020aaa9d87eccc6d051b3,456bdaea194b7b47db8156a59e4da2c5,11.9


In [20]:
d= b.groupby(["user_id","video_id"]).agg({'weighted_score':['sum']}).unstack(level=1).reset_index()
d.columns=d.columns.get_level_values(0)

d1= b1.groupby(["user_id","video_id"]).agg({'weighted_score':['sum']}).unstack(level=1).reset_index()
d1.columns=d1.columns.get_level_values(0)

In [21]:
d.head(5)

,user_id,weighted_score,weighted_score,weighted_score,weighted_score,weighted_score,weighted_score,weighted_score,weighted_score,weighted_score,...,weighted_score,weighted_score,weighted_score,weighted_score,weighted_score,weighted_score,weighted_score,weighted_score,weighted_score,weighted_score
0,0005266eb37020aaa9d87eccc6d051b3,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,NaN,...,0.0,0.0,NaN,0.0,0.0,7.6,0.0,0.0,NaN,0.0
1,00283999ebb74c7052f962400b51af1b,5.7,0.0,12.6,42.1,72.6,5.7,0.0,41.1,19.5,...,7.6,1735.6,19.9,0.0,102.2,0.0,13.7,7.6,24.5,28.3
2,003cb7ee2f970b407b0087075b41b608,NaN,NaN,NaN,NaN,0.0,NaN,NaN,28.3,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,26.9,NaN,NaN
3,004cab314f7310363f7a802369867f9e,12.6,45.6,NaN,26.4,0.0,NaN,19.2,29.1,38.7,...,56.4,63.4,NaN,19.5,NaN,0.0,NaN,0.0,10.7,0.0
4,006c65a0946666507fd85ad824ff5909,NaN,NaN,12.6,39.8,85.1,12.6,NaN,111.9,NaN,...,14.9,21.8,12.6,7.6,43.3,7.6,12.6,12.6,NaN,58.2


In [22]:
d.columns =['user_id','V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','V29','V30','V31','V32','V33','V34','V35','V36','V37','V38','V39','V40','V41','V42','V43','V44','V45','V46','V47','V48','V49','V50','V51','V52','V53','V54','V55','V56','V57','V58','V59','V60','V61','V62','V63']
d1.columns =['user_id','V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','V29','V30','V31','V32','V33','V34','V35','V36','V37','V38','V39','V40','V41','V42','V43','V44','V45','V46','V47','V48','V49','V50','V51','V52','V53','V54','V55','V56','V57','V58','V59','V60','V61','V62','V63']

In [23]:
d.head(5)

,user_id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63
0,0005266eb37020aaa9d87eccc6d051b3,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,NaN,...,0.0,0.0,NaN,0.0,0.0,7.6,0.0,0.0,NaN,0.0
1,00283999ebb74c7052f962400b51af1b,5.7,0.0,12.6,42.1,72.6,5.7,0.0,41.1,19.5,...,7.6,1735.6,19.9,0.0,102.2,0.0,13.7,7.6,24.5,28.3
2,003cb7ee2f970b407b0087075b41b608,NaN,NaN,NaN,NaN,0.0,NaN,NaN,28.3,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,26.9,NaN,NaN
3,004cab314f7310363f7a802369867f9e,12.6,45.6,NaN,26.4,0.0,NaN,19.2,29.1,38.7,...,56.4,63.4,NaN,19.5,NaN,0.0,NaN,0.0,10.7,0.0
4,006c65a0946666507fd85ad824ff5909,NaN,NaN,12.6,39.8,85.1,12.6,NaN,111.9,NaN,...,14.9,21.8,12.6,7.6,43.3,7.6,12.6,12.6,NaN,58.2


In [24]:
d.fillna(0)
d1 = d1.fillna(0)

In [25]:
passfail = pd.read_csv("/Users/nsehgal/Documents/Data Mining/dataset/TrainLabel.csv")

In [26]:
pass_fail = pd.DataFrame(passfail)
pass_fail.head(5)

,user_id,grade
0,bd4c9ab730f5513206b999ec0d90d1fb,0
1,d2a27e83d429f0dcae6b937cf440aeb1,0
2,a376033f78e144f494bfc743c0be3330,0
3,2a0f97f81755e2878b264adf39cba68e,0
4,60ad83801910ec976590f69f638e0d6d,0


In [28]:
#Train Labels are merged with Train Features
data= pd.merge(d, pass_fail, on='user_id')
data.head(5)
data.fillna(0)

,user_id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,grade
0,0005266eb37020aaa9d87eccc6d051b3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.6,0.0,0.0,0.0,0.0,0
1,00283999ebb74c7052f962400b51af1b,5.7,0.0,12.6,42.1,72.6,5.7,0.0,41.1,19.5,...,1735.6,19.9,0.0,102.2,0.0,13.7,7.6,24.5,28.3,1
2,003cb7ee2f970b407b0087075b41b608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.3,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,26.9,0.0,0.0,1
3,004cab314f7310363f7a802369867f9e,12.6,45.6,0.0,26.4,0.0,0.0,19.2,29.1,38.7,...,63.4,0.0,19.5,0.0,0.0,0.0,0.0,10.7,0.0,0
4,006c65a0946666507fd85ad824ff5909,0.0,0.0,12.6,39.8,85.1,12.6,0.0,111.9,0.0,...,21.8,12.6,7.6,43.3,7.6,12.6,12.6,0.0,58.2,0
5,0083870f576cd563ee5b19c97ef77a2c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,00a7076382b9970980e540230e0bf61f,6.9,0.0,0.0,0.0,40.2,0.0,0.0,0.0,5.7,...,12.6,0.0,0.0,33.4,0.0,0.0,16.9,5.7,5.7,1
7,00ab1656a2996a97029c3fa7a7010498,108.8,198.5,173.7,190.9,361.3,176.3,92.7,103.4,189.7,...,390.3,194.3,124.5,267.5,22.9,167.9,297.9,164.4,195.9,1
8,00afe3e30d052142dc018ef0e9ca7b30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,00c8744837dba943b3f1e8bad3791ab4,0.0,0.0,0.0,0.0,14.5,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,7.6,0.0,0.0,0.0,0.0,0


In [36]:
#Python Libraries
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap

import pandas as pd
import numpy as np

np.random.seed(2017)
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [37]:
features = list(data.columns[1:64])
X = data[features]
y = data['grade']
X=X.fillna(0)

In [38]:
Xtest = d1.iloc[:,1:]
Xtest = Xtest.fillna(0)

In [39]:
X1 = Xtest

In [40]:
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn import metrics
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

xx_train, xx_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2017)

In [41]:
LR = LogisticRegression(random_state=2017)
RF = RandomForestClassifier(n_estimators = 100, random_state=2017)
SVM = SVC(random_state=0, probability=True)
KNC = KNeighborsClassifier()
DTC = DecisionTreeClassifier()
ABC = AdaBoostClassifier(n_estimators = 100)
BC = BaggingClassifier(n_estimators = 100)
GBC = GradientBoostingClassifier(n_estimators = 100)

In [42]:
clfs = []
print('5-fold cross validation:\n')
for clf, label in zip([LR, GBC, SVM, KNC, DTC, ABC, BC, RF], 
                      ['Logistic Regression', 
                       'Gradient Boosting', 
                       'Support Vector Machine',
                       'KNeighbors',
                       'Decision Tree',
                       'Ada Boost',
                       'Bagging',
                       'Random Forest']):
    scores = cross_validation.cross_val_score(clf, xx_train, y_train, cv=5, scoring='accuracy')
    print("Train CV Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
    md = clf.fit(xx_train, y_train)    
    clfs.append(md)
    print("Test Accuracy: %0.2f " % (metrics.accuracy_score(clf.predict(xx_test), y_test)))   

5-fold cross validation:

Train CV Accuracy: 0.84 (+/- 0.01) [Logistic Regression]
Test Accuracy: 0.85 
Train CV Accuracy: 0.88 (+/- 0.00) [Gradient Boosting]
Test Accuracy: 0.89 
Train CV Accuracy: 0.85 (+/- 0.00) [Support Vector Machine]
Test Accuracy: 0.87 
Train CV Accuracy: 0.85 (+/- 0.01) [KNeighbors]
Test Accuracy: 0.87 
Train CV Accuracy: 0.83 (+/- 0.01) [Decision Tree]
Test Accuracy: 0.83 
Train CV Accuracy: 0.86 (+/- 0.01) [Ada Boost]
Test Accuracy: 0.88 
Train CV Accuracy: 0.87 (+/- 0.01) [Bagging]
Test Accuracy: 0.87 
Train CV Accuracy: 0.88 (+/- 0.01) [Random Forest]
Test Accuracy: 0.90 


In [43]:
#Random Forest perform best, so we choose random forest to predict test data labels
Xteststand = scaler.transform(X1)

In [44]:
predictions = clf.predict(Xteststand)

In [45]:
len(predictions)

1293

In [46]:
predictions

array([0, 0, 0, ..., 0, 1, 0])

In [47]:
abc =pd.DataFrame(predictions)

In [48]:
abc.to_csv('out.csv')